In [1]:
import torch
print("Broj GPU-ova:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"{i}: {torch.cuda.get_device_name(i)}")


Broj GPU-ova: 4
0: Tesla V100-SXM2-16GB
1: Tesla V100-SXM2-16GB
2: Tesla V100-SXM2-16GB
3: Tesla V100-SXM2-16GB


# Model distilroberta-base

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer
from datasets import load_from_disk
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

In [3]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [4]:
train_dataset = load_from_disk("tokenized/train")
val_dataset = load_from_disk("tokenized/val")
test_dataset = load_from_disk("tokenized/test")

train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")

In [ ]:
model_name = "distilroberta-base"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  
    ignore_mismatched_sizes=True
)


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

2025-10-26 18:20:06.553934: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-26 18:20:06.613339: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-26 18:20:08.059897: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./distilroberta-base",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs2",
    logging_steps=100
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipykernel_128817/2796941932.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.152200,0.143286,0.946494,0.946493,0.946521,0.946494
2,0.135600,0.135279,0.950119,0.950119,0.950119,0.950119
3,0.124700,0.133478,0.951719,0.951719,0.951720,0.951719
4,0.115700,0.132320,0.952139,0.952138,0.952157,0.952139
5,0.107300,0.133385,0.952780,0.952780,0.952782,0.952780


/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=31645, training_loss=0.13400308696706442, metrics={'train_runtime': 15188.4358, 'train_samples_per_second': 1066.597, 'train_steps_per_second': 2.083, 'total_flos': 5.3649097754582016e+17, 'train_loss': 0.13400308696706442, 'epoch': 5.0})

In [12]:
metrics = trainer.evaluate(test_dataset)
print(metrics)


/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.13477495312690735, 'eval_accuracy': 0.9514748786871967, 'eval_f1': 0.9514745726661229, 'eval_precision': 0.9514862557696173, 'eval_recall': 0.9514748786871967, 'eval_runtime': 161.9118, 'eval_samples_per_second': 2470.474, 'eval_steps_per_second': 4.83, 'epoch': 5.0}


In [13]:
trainer.save_model("./distilroberta-base")
tokenizer.save_pretrained("./distilroberta-base")


('./distilroberta-base/tokenizer_config.json',
 './distilroberta-base/special_tokens_map.json',
 './distilroberta-base/vocab.json',
 './distilroberta-base/merges.txt',
 './distilroberta-base/added_tokens.json')